In [8]:
import getpass
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

os.environ["LANGCHAIN_TRACING_V2"] = "true"

load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangChain API key: ")

model = init_chat_model(
    "openai:gpt-4o",
    temperature=0.7
)

In [9]:
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END

def get_weather(location: str):
    """Call to get the current weather"""
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy"
    else: 
        return "It's 90 degrees and sunny"

tool_node = ToolNode([get_weather]) # Define a tool node that can be called by the LLM

model_with_tools = model.bind_tools([get_weather])

def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls: # Checks if the last message from the LLM included any tool calls.
        return "tools"
    return END

def call_model(state: MessagesState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_node("tools", tool_node)

builder.add_edge(START, "call_model")
builder.add_conditional_edges("call_model", should_continue, ["tools", END])
builder.add_edge("tools", "call_model")

graph = builder.compile()

graph.invoke({"messages": [{"role": "user", "content": "what's the weather in sf?"}]})


{'messages': [HumanMessage(content="what's the weather in sf?", additional_kwargs={}, response_metadata={}, id='cff3f852-3643-4d1a-8fcd-785f77018b69'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2r4BeWXWOdoFhFSQrywJ8DC6', 'function': {'arguments': '{"location":"San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 51, 'total_tokens': 66, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'id': 'chatcmpl-Bs2W868yJ5JPqoaxdEgIrWcjLX6bo', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--538cc413-1cb0-4815-95ac-fa40b7e0083f-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'San Francisco'